In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')
import session

from aocd import get_data

In [557]:
data = get_data(day=4).splitlines()
draws= np.array(list(map(int, data.pop(0).split(','))))
boards = np.array([line.split() for line in data if len(line) > 0]).reshape(-1, 5, 5).astype(int)

In [558]:
%%timeit

results = np.zeros(boards.shape)
mask = np.ones(boards.shape[0]).astype(bool)
wins = []

for n in draws:

    results = results + (temp == n)

    row_sum = (results.sum(axis=2) == 5).any(axis=1)
    col_sum = (results.sum(axis=1) == 5).any(axis=1)

    total = mask & (row_sum | col_sum)

    if total.any():

        mask[total] = False
        score = (temp[total] * (1-results[total])).sum(axis=(1,2)) * n
        wins = wins + list(score)

#print(f'1 : {wins[0]}')
#print(f'2 : {wins[-1]}')

8.95 ms ± 38.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [565]:
%%timeit

shape = (len(draws), boards.shape[0], boards.shape[1], boards.shape[2])
results = np.zeros(shape)
boards_in_time = np.repeat(boards, 100, axis=0).reshape(shape).swapaxes(0,1)
draws_in_time = draws.repeat(100*5*5).reshape(shape)

comparision_in_time = (boards_in_time == draws_in_time)
cumulative_play = np.cumsum(comparision_in_time, axis=0)

board_win = ((cumulative_play.sum(axis=2) == 5) | (cumulative_play.sum(axis=3) == 5)).any(axis=2)
new_wins = np.diff(board_win, axis=0, prepend=False)

first_win = new_wins.any(axis=1)
win_rounds = np.arange(len(draws))[first_win]

first_win_round = win_rounds[0]
first_win_board = new_wins[first_win_round].argmax()

last_win_round = win_rounds[-1]
last_win_board = new_wins[last_win_round].argmax()

p1 = (boards[first_win_board] * (1-cumulative_play[first_win_round, first_win_board])).sum() * draws[first_win_round]
p2 = (boards[last_win_board] * (1-cumulative_play[last_win_round, last_win_board])).sum() * draws[last_win_round]

#print(p1, p2)

4.51 ms ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
